In [1]:
import pandas as pd
import csv
import os
import time
from collections import defaultdict
import numpy as np
import itertools
import db_structure
import networkx as nx
import matplotlib.pyplot as plt
import utilities as u
from flask import jsonify
import logging
# import exceptions
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from web.models import *
from web import db

logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
%load_ext autoreload
%aimport -web
%autoreload 2

In [ ]:
dataset_name = 'SCA_AcuteCare'
draw_db = db_structure.DBExtractor(dataset_name)

G = nx.DiGraph()
table_names = [x.table_name for x in db.session.query(TableMetadata).filter(TableMetadata.dataset_name == dataset_name, TableMetadata.num_records >= 10000).all()]
G.add_nodes_from(table_names)
for table_name in table_names:
    for sibling in draw_db.find_table_siblings(table_name):
        if sibling in table_name:
            G.add_edge(table_name, sibling)
            G.add_edge(sibling, table_name)
    for child in draw_db.find_table_children(table_name):
        if child in table_names:
            G.add_edge(table_name, child)
plt.figure(figsize=(10,10))

#draw with networkx built-ins
nx.draw_networkx(G, node_shape="None", width=0.2)

#OR

#draw using pygraphviz
#A = nx.nx_agraph.to_agraph(G)
#H = nx.nx_agraph.from_agraph(A)
#nx.draw_spring(H, node_shape="None", with_labels=True, width=0.2)

In [4]:
x = db_structure.DBMaker(dataset_name='TOPICC', directory_path='datasets\\TOPICC')
x.create_db_metadata(dump_to_data_db=True)

In [3]:
db_structure.DBDestroyer('TOPICC').remove_db()

In [ ]:
y = db_structure.DBLinker(dataset_name='TOPICC')

In [ ]:
y.add_global_fk('PudID')

In [ ]:
z = db_structure.DBExtractor(dataset_name='TOPICC')

In [ ]:
path = ['HOSPITALADMIT', 'CAREPROCESSES', 'DEATH']
start = time.time()
df = z.get_df_from_path(path, table_columns_of_interest=[('HOSPITALADMIT', 'PudID'), ('HOSPITALADMIT', 'Sex'), ('CAREPROCESSES', 'MechVent'), ('DEATH', 'DeathMode')])
end = time.time()
print(end-start)

In [ ]:
df

In [51]:
x = db_structure.DBMaker(dataset_name='SCA_AcuteCare', sql_server='CYKPANADBSQL', sql_db='SCA_AcuteCare', schema_name='dbo')
x.create_db_metadata(ignore_tables_with_substrings=['MOTempRecsGrp', 'MO_MeasureInfo', 'MO_MedicationDim_55_'])

C:\Users\SShah50\AppData\Roaming\Python\Python36\site-packages\dateutil\parser\_parser.py:1218: UnknownTimezoneWarning: tzname M identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [75]:
y = db_structure.DBLinker(dataset_name='SCA_AcuteCare')
for i in y.get_common_column_names():
    if i.upper()[-2:] == 'ID' and i.upper() not in ['GUID']:
        print(i)
        y.add_global_fk(i)

APMInsuranceCarrierDimID
AbnormalityDimID
AccommodationDimID
AccountDimID
AccountTypeDimID
ActionCodedReasonTypeDimID
ActionReasonDimID
ActionTypeDimID
AdmitDateDimID
AdmitReasonDimID
AdmitShiftDimID
AdmitSourceDimID
AdmitTypeDimID
AgeDimID
AgeRangeItemID
AgeRangeSetID
AlertEventTypeDimID
AlertNameDimID
AlertTypeDimID
AllergenDimID
AllergyCategoryTypeDimID
AllergyTypeDimID
AmbulStatusDimID
AncillaryFacilityDimID
ApplicSourceDimID
AppointmentCategoryDimID
ArrivalDateDimID
AuthoredProviderDimID
BillingAddressDimID
BlockDefinitionDimID
CareLevelDimID
CatalogItemTaskGUID
CategoryCodeDimID
ChartGUID
ClientDocumentGUID
ClientGUID
ClientUserDataGUID
ClientVisitGUID
ClinDecSupportDescDimID
ClinDecSupportPriorityDimID
CommentDeclarationTypeDimID
ConfidenceDimID
CostDimID
CurrentLocationGUID
DemographicDimID
DestinationTypeDimID
DiagDimID
DiagTypeDimID
DiagnosisGroupID
DiagnosisID
DictionaryID
DischargeDateDimID
DischargeDispositionDimID
DischargeLocationDimID
DischargeServiceDimID
DischargeShif

In [50]:
db_structure.DBDestroyer('SCA_AcuteCare').remove_db()

In [85]:
z = db_structure.DBExtractor('SCA_AcuteCare')

In [86]:
paths = [['BF_FoleyTimes', 'SCAVisit']]
table_columns_of_interest = [('BF_FoleyTimes', 'FoleyDays'), ('SCAVisit', 'IsCHF')]
filters_with_name_keys = {'SCAVisit_IsCHF': {'filter': ['0', '1'], 'type': 'bool'}}
groupby_columns = ['SCAVisit_IsCHF']
aggregate_column = 'BF_FoleyTimes_FoleyDays'
aggregate_fxn = 'Mean'
df = z.get_biggest_df_from_paths(paths, table_columns_of_interest, limit_rows=10000)

In [87]:
agg_df = z.aggregate_df(df, groupby_columns, filters_with_name_keys, aggregate_column, aggregate_fxn)

In [88]:
agg_df

,BF_FoleyTimes_FoleyDays,groupby_labels
0,3.22,0
1,6.49,1
